LR Finder

purpose: 

find the best learning rates for the lstm head and the resnet body in an efficient manner without having to iterate through a manual list of learning rates

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from model import DeepfakeDetector
import torchvision.models as models
from ignite.handlers import FastaiLRFinder
from ignite.engine import Engine
from dataset import videos_dataset_creation, DeepFakeDataset, DataLoader, ROOT_DIR, REAL_FOLDER, FAKE_FOLDER
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = DeepfakeDetector(
  resnet_weights=models.ResNet34_Weights.DEFAULT,
  resnet_prog=True,
  input_size=512,
  hidden_size=512,
  num_layers=2,
  num_classes=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=0.001)



Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to C:\Users\andre/.cache\torch\hub\checkpoints\resnet34-b627a593.pth


100%|██████████| 83.3M/83.3M [00:04<00:00, 21.5MB/s]


In [ ]:
#compose for data preprocessing
compose = transforms.Compose([
  transforms.Resize(224),
  transforms.ToTensor(),
  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

labels = [(REAL_FOLDER, 0), (FAKE_FOLDER, 1)]

#create list of tuples of videos
videos_dataset = videos_dataset_creation(ROOT_DIR, labels)

#creating and splitting datasets
train_ratio, val_ratio = int(len(videos_dataset) * 0.70), int(len(videos_dataset) * 0.15)
train_split, val_split, test_split = videos_dataset[:train_ratio], videos_dataset[train_ratio:train_ratio+val_ratio], videos_dataset[train_ratio+val_ratio:]

train_dataset = DeepFakeDataset(20, ROOT_DIR, train_split, transforms=compose)
val_dataset = DeepFakeDataset(20, ROOT_DIR, val_split, transforms=compose)
test_dataset = DeepFakeDataset(20, ROOT_DIR, test_split, transforms=compose)


#Dataloaders for all dataset splits
train_loader = DataLoader(
  train_dataset,
  batch_size=32,
  shuffle=True,
  num_workers=2,
  pin_memory=True,
  drop_last=True
)

val_loader = DataLoader(
  val_dataset,
  batch_size=32,
  shuffle=False,
  num_workers=2,
  pin_memory=True
)

test_loader = DataLoader(
  test_dataset,
  batch_size=32,
  shuffle=False,
  num_workers=2,
  pin_memory=True
)

In [3]:
def update_model(engine, batch):
  images, labels = batch
  images = images.to(device)
  labels = labels.to(device)

  optimizer.zero_grad()
  outputs = model(images)
  loss = criterion(outputs, labels)

  loss.backward()
  optimizer.step()

  return loss.item()

In [ ]:
trainer = Engine(update_model)
lr_finder = FastaiLRFinder()
to_save = {"model" : model, "optimizer" : optimizer}

with lr_finder.attach(trainer, to_save=to_save) as trainer_with_lr_finder:
  trainer_with_lr_finder.run(train_loader)

lr_finder.get_results()

lr_finder.plot()

lr_finder.lr_suggestion()

